In [50]:
# Imports
from pandas import json_normalize
import requests
import json
import os
import re
import time
import fnmatch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io import sql


In [60]:
# Read file into dataframe
df_tweets = pd.read_csv('../01_Kontext/10_datasource/cyberbullying_tweets.csv', sep=',', encoding='utf-8')
print(df_tweets.shape)
df_tweets.head(5)

(47692, 2)


,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [52]:
df_tweets['cyberbullying_type'].unique()

array(['not_cyberbullying', 'other_cyberbullying', 'none', 'gender', nan,
       'sexism', 'age', 'ethnicity', 'religion'], dtype=object)

In [53]:
# Change strings to uppercase 
df_tweets['tweet_text'] = df_tweets['tweet_text'].str.upper()
print(df_tweets['tweet_text'].head(10), '\n')

0                  @G56YU WHAT IS THAT?
1                      @STOCKPUTOUT YES
2                               5/13/15
3            @BLACKAMAZON EXACTLY THIS.
4    @ODDTANKOUT HTTP://T.CO/K5MMJE8UME
5                                  ROFL
6              OH, YES HE IS A BAD GUY.
7                HTTP://T.CO/NXGTQSLCHG
8                             ¯\_(ツ)_/¯
9                        GAY RAPE JOKE?
Name: tweet_text, dtype: object 



In [54]:
# Create a pattern which can be used to search the variable 'tweet_text'
patternHarass = '(INSULT)|(ABUSE)|(THREATEN)|(VICTIM)|(TORMENT)|(PERSECUTE)|(INTIMIDATE)|(AGGRESSION)|(RELENTLESS)|(BULLYING)|(HOSTILITY)|(SLANDER)|(HURTFUL)|(ATTACKS)|(OFFENSE)|(MALICIOUS)|(STALKER)|(TAUNT)|(ABUSIVE)|(TARGET)'
patternCyberstalk = '(STALK)|(PURSUE)|(OBSESS)|(WATCH)|(FOLLOW)|(INTRUDE)|(SPY)|(UNWANTED)|(MONITORING)|(SURVEILLANCE)|(TRACKING)|(INVASION)|(SHADOW)|(PRYING)|(PERSISTENCE)|(STALKING)|(SPYING)|(INTRUSIVE)|(PERVASION)|(PURSUIT)'
patternFlaming = '(PROVOCATION)|(INSULTING)|(AGGRESSIVE)|(HOSTILE)|(OFFEND)|(HEATED)|(ARGUMENTATIVE)|(OFFENSIVE)|(CONTENTIOUS)|(DISPUTE)|(ANTAGONISTIC)|(FIERY)|(INCENDIARY)|(HARSH)|(COMBATIVE)|(CLASHING)|(ARGUMENT)|(PROVOCATIVE)|(SPARRING)|(CONTROVERSIAL)'
patternOuting = '(EXPOSURE)|(REVEAL)|(DISCLOSURE)|(BETRAYAL)|(UNMASKING)|(UNVEILING)|(LEAKAGE)|(PUBLICATION)|(BETRAYED)|(UNCOVERING)|(EXPOSED)|(CONFESSION)|(REVEALING)|(LEAKED)|(PRIVACY)|(PRIVATE)|(DATA)|(DETAILS)|(IDENTIFY)|(PUBLISH)'
patternDoxing = '(DOX)|(INFORMATION)|(REVEAL)|(DATA)|(PERSONAL)|(IDENTITY)|(PRIVACY)|(EXPOSED)|(PUBLISH)|(UNMASKING)|(LEAKED)|(PROFILE)|(ADDRESS)|(DETAILS)|(IDENTIFY)|(PUBLISH)|(REVEALING)|(EXPOSE)|(DATA)|(UNVEILING)'
patternExclusion = '(ISOLATE)|(OSTRACIZE)|(EXCLUDE)|(ALIENATION)|(SHUN)|(REJECT)|(IGNORED)|(OUTSIDER)|(SOLITUDE)|(SEGREGATE)|(ABANDONMENT)|(NEGLECTED)|(ISOLATED)|(MARGINALIZED)|(UNWANTED)|(LONELINESS)|(REJECTION)|(ISOLATION)|(NEGLECT)|(NEGLECTING)'
patternImpersonation = '(FAKE)|(IDENTITY)|(DECEPTION)|(FRAUDULENT)|(PRETEND)|(IMITATE)|(COUNTERFEIT)|(MIMIC)|(PHONY)|(FRAUD)|(FORGERY)|(IMPOSTER)|(DECEIVER)|(FAKE PROFILE)|(FALSE IDENTITY)|(IDENTITY THEFT)|(FRAUDULENT ACCOUNT)|(MIMICKING)|(FORGED)|(COUNTERFEITING)'
patternTrolling = '(TROLL)|(PROVOCATION)|(AGITATION)|(DISRUPTION)|(ANNOYING)|(NUISANCE)|(IRRITATION)|(HARASSMENT)|(MISCHIEF)|(TEASING)|(INSTIGATION)|(OFFENSIVE)|(MOCKERY)|(RIDICULE)|(TAUNT)|(DISTURBANCE)|(BAITING)|(ANTAGONISTIC)|(CONTROVERSIAL)|(PRANKS)'
patternHateSpeech = '(HATE)|(BIGOTRY)|(DISCRIMINATION)|(PREJUDICE)|(STEREOTYPING)|(HOMOPHOBIA)|(MISOGYNY)|(RACIAL)|(SLURS)|(XENOPHOBIA)|(BIAS)|(OFFENSIVE)|(DEROGATORY)|(VILE)|(PREJUDICED)|(HATE-FILLED)|(INTOLERANCE)|(RACIST)|(SEXIST)|(HOMOPHOBIC)'
patternThreats = '(THREATEN)|(VIOLENCE)|(DANGER)|(MENACE)|(INTIMIDATE)|(HARM)|(PERIL)|(WARNING)|(COERCION)|(FEAR)|(HARBINGER)|(TERRORIZE)|(MENACING)|(ASSAULT)|(HOSTILITY)|(AGGRESSION)|(THREATENING)|(THREATENED)|(ABUSIVE)|(VIOLENT)'

# Create new variable 'luxurious' as binary dummy (0/1) variable
df_tweets['harass'] = df_tweets['tweet_text'].str.contains(pat = patternHarass).astype(float)
print(df_tweets['harass'].sum())
df_tweets['cyberstalk'] = df_tweets['tweet_text'].str.contains(pat = patternCyberstalk).astype(float)
print(df_tweets['cyberstalk'].sum())
df_tweets['flaming'] = df_tweets['tweet_text'].str.contains(pat = patternFlaming).astype(float)
print(df_tweets['flaming'].sum())
df_tweets['outing'] = df_tweets['tweet_text'].str.contains(pat = patternOuting).astype(float)
print(df_tweets['outing'].sum())
df_tweets['doxing'] = df_tweets['tweet_text'].str.contains(pat = patternDoxing).astype(float)
print(df_tweets['doxing'].sum())
df_tweets['exclusion'] = df_tweets['tweet_text'].str.contains(pat = patternExclusion).astype(float)
print(df_tweets['exclusion'].sum())
df_tweets['impersonation'] = df_tweets['tweet_text'].str.contains(pat = patternImpersonation).astype(float)
print(df_tweets['impersonation'].sum())
df_tweets['trolling'] = df_tweets['tweet_text'].str.contains(pat = patternTrolling).astype(float)
print(df_tweets['trolling'].sum())
df_tweets['hateSpeech'] = df_tweets['tweet_text'].str.contains(pat = patternHateSpeech).astype(float)
print(df_tweets['hateSpeech'].sum())
df_tweets['threats'] = df_tweets['tweet_text'].str.contains(pat = patternThreats).astype(float)
print(df_tweets['threats'].sum())

# Show values
df_tweets[['tweet_text','harass','cyberstalk','flaming','outing', 'doxing', 'exclusion', 'impersonation', 'trolling', 'hateSpeech', 'threats']]

0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0


/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_41996/403006207.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['harass'] = df_tweets['tweet_text'].str.contains(pat = patternHarass).astype(float)
/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_41996/403006207.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['cyberstalk'] = df_tweets['tweet_text'].str.contains(pat = patternCyberstalk).astype(float)
/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_41996/403006207.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['flaming'] = df_tweets['tweet_text'].str.contains(pat = patternFlaming).astype(float)
/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykerne

,tweet_text,harass,cyberstalk,flaming,outing,doxing,exclusion,impersonation,trolling,hateSpeech,threats
0,@G56YU WHAT IS THAT?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,@STOCKPUTOUT YES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5/13/15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,@BLACKAMAZON EXACTLY THIS.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,@ODDTANKOUT HTTP://T.CO/K5MMJE8UME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
95,SOON ™,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,ANNIE IS COATED IN + 1000 SUNSCREEN. #MKR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,NIGHTS MKR PPL :D #MKR #MKR2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,FINALLY HOME.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
df_tweets.dtypes

interaction_id             int64
tweet_id                   int64
senderuser_id              int64
receiveruser_id            int64
tweet_text                object
cyberbullying_type        object
interaction_timestamp     object
harass                   float64
cyberstalk               float64
flaming                  float64
outing                   float64
doxing                   float64
exclusion                float64
impersonation            float64
trolling                 float64
hateSpeech               float64
threats                  float64
dtype: object

In [56]:
df_tweets['cyberbullying_type'].unique()

array(['not_cyberbullying', 'other_cyberbullying', 'none', 'gender', nan,
       'sexism', 'age', 'ethnicity', 'religion'], dtype=object)

In [57]:
df_tweets.to_csv('dataset_tweets_featured.csv',
                      sep=",",
                      encoding='utf-8',
                      index=False)